# Fine-Tuned Facial Emotion Recognition with Transfer Learning (VGG16)
Uses transfer learning, advanced augmentation, and best practices for improved FER2013 accuracy.

**Steps:** Install packages, upscale images to 224x224 and convert grayscale to RGB, use VGG16, train with callbacks, save best model, and test in real time.

In [ ]:
# 1. Install required libraries
!pip install tensorflow keras opencv-python numpy matplotlib pandas scikit-learn --quiet


In [2]:
# 2. Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np, cv2, os, matplotlib.pyplot as plt, datetime
print('TensorFlow version:', tf.__version__)



TensorFlow version: 2.15.0


In [3]:
batch_size = 64
def gray2rgb_resize(img):
    import cv2
    # img is 2D (h, w, 1)
    if img.shape[-1] == 1:
        img = img.squeeze(-1)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (224, 224))
    return img

train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, shear_range=0.2, horizontal_flip=True, validation_split=0.2,
    preprocessing_function=gray2rgb_resize
)

train_gen = train_aug.flow_from_directory(
    'data/train',
    target_size=(224, 224),   # Change from (48,48) to (224,224)
    color_mode='rgb',         # Change from 'grayscale' to 'rgb'
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

val_gen = train_aug.flow_from_directory(
    'data/train',
    target_size=(224, 224),   # Change here as well
    color_mode='rgb',         # Change here as well
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

steps_train = train_gen.samples // batch_size
steps_val = val_gen.samples // batch_size


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [4]:
# 4. Load pre-trained VGG16 for transfer learning
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in vgg.layers[:-4]: layer.trainable = False  # fine-tune only last 4 conv layers
x = vgg.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=preds)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [5]:
# 5. Training callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1),
    ModelCheckpoint('best_emotion_vgg16.h5', save_best_only=True, monitor='val_loss', verbose=1)
]
epochs = 30
history = model.fit(
    train_gen,
    steps_per_epoch=steps_train,
    validation_data=val_gen,
    validation_steps=steps_val,
    epochs=epochs,
    callbacks=callbacks
)



Epoch 1/30


358/358 [==============================] - ETA: 0s - loss: 1.6270 - accuracy: 0.3686
Epoch 1: val_loss improved from inf to 1.51889, saving model to best_emotion_vgg16.h5
358/358 [==============================] - 1796s 5s/step - loss: 1.6270 - accuracy: 0.3686 - val_loss: 1.5189 - val_accuracy: 0.4055 - lr: 1.0000e-04
Epoch 2/30


c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


358/358 [==============================] - ETA: 0s - loss: 1.3464 - accuracy: 0.4859
Epoch 2: val_loss improved from 1.51889 to 1.31844, saving model to best_emotion_vgg16.h5
358/358 [==============================] - 1653s 5s/step - loss: 1.3464 - accuracy: 0.4859 - val_loss: 1.3184 - val_accuracy: 0.4921 - lr: 1.0000e-04
Epoch 3/30
358/358 [==============================] - ETA: 0s - loss: 1.2469 - accuracy: 0.5290
Epoch 3: val_loss improved from 1.31844 to 1.26802, saving model to best_emotion_vgg16.h5
358/358 [==============================] - 1647s 5s/step - loss: 1.2469 - accuracy: 0.5290 - val_loss: 1.2680 - val_accuracy: 0.5200 - lr: 1.0000e-04
Epoch 4/30
358/358 [==============================] - ETA: 0s - loss: 1.1865 - accuracy: 0.5521
Epoch 4: val_loss improved from 1.26802 to 1.24708, saving model to best_emotion_vgg16.h5
358/358 [==============================] - 1656s 5s/step - loss: 1.1865 - accuracy: 0.5521 - val_loss: 1.2471 - val_accuracy: 0.5195 - lr: 1.0000e-04
Epo

# Notes
- Training may take significant GPU time.
- Will likely increase validation accuracy substantially over base CNN.
